In [2]:
import osmium
import csv
from geopy.distance import geodesic


In [ ]:

MIN_DISTANCE = 50  # meters for ways


In [4]:

class NamedHandler(osmium.SimpleHandler):
    def __init__(self, outfile):
        super().__init__()
        self.writer = csv.writer(open(outfile, "w", newline=""))
        self.writer.writerow(["name", "lat", "lon"])

    def node(self, n):
        # Node with a name
        name = n.tags.get("name")
        if name:
            self.writer.writerow([name, int(n.location.lat*1e6), int(n.location.lon*1e6)])

    def way(self, w):
        # Way with a name
        name = w.tags.get("name")
        if name:
            last_point = None
            for n in w.nodes:
                lat, lon = n.lat, n.lon
                if last_point is None or geodesic((last_point[0], last_point[1]), (lat, lon)).meters >= MIN_DISTANCE:
                    self.writer.writerow([name, int(lat*1e6), int(lon*1e6)])
                    last_point = (lat, lon)


In [7]:

handler = NamedHandler("saved_csv/p3_named_everything.csv")
handler.apply_file("kanpur_pois.pbf", locations=True)
